## 1. Framing the problem

The objective is to predict the price of cars based on its properties and characteristics. The solution will be used to predict the price of new cars, as well as to understand which factors are detrimental to a car's price. 

The prediction model will be a supervised, regression model since the model will predict the value of popularity, the target variable, from the other variables contained in the dataset. The performance of the model will be evaluated using Mean Squared Error (MSE) to get a sense on how the model accuracy.

The target is to minimize the value of MSE as near as it can get to 0. The closer the MSE to 0, the more accurate the model is. Having an accurate model will allow us to predict the price precisely.

## 2. Data Acquisition

Checking python and scikit-learn packages, importing packages, and setting seed

In [1]:
# Python ≥ 3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥ 0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Importing packages
import pandas as pd
import numpy as np
import pandas_profiling
from sklearn.ensemble import IsolationForest
import rfpimp
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error

import random
random.seed(0)

D:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


Importing the data from .csv
file

In [2]:
car_df = pd.read_csv('D:\Python\CarPrice_Assignment.csv')

## 3. Data Exploration

Preview of the dataset

In [3]:
car_df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


Generate the profile report, containing various and extensive information, statistics, and histograms of the each variable for the data exploration task. 

In [4]:
car_df.profile_report(html={'style':{'full_width':True}})

# To generate a HTML report
profile = car_df.profile_report(title='Car_Train Profiling Report')
profile.to_file(output_file="car_train.html")

## 4. Data Preparation

Generating correlation matrix

In [5]:
car_df_corr = car_df.corr()
car_df_corr

,car_ID,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
car_ID,1.000000,-0.151621,0.129729,0.170636,0.052387,0.255960,0.071962,-0.033930,0.260064,-0.160824,0.150276,-0.015006,-0.203789,0.015940,0.011255,-0.109093
symboling,-0.151621,1.000000,-0.531954,-0.357612,-0.232919,-0.541038,-0.227691,-0.105790,-0.130051,-0.008735,-0.178515,0.070873,0.273606,-0.035823,0.034606,-0.079978
wheelbase,0.129729,-0.531954,1.000000,0.874587,0.795144,0.589435,0.776386,0.569329,0.488750,0.160959,0.249786,0.353294,-0.360469,-0.470414,-0.544082,0.577816
carlength,0.170636,-0.357612,0.874587,1.000000,0.841118,0.491029,0.877728,0.683360,0.606454,0.129533,0.158414,0.552623,-0.287242,-0.670909,-0.704662,0.682920
carwidth,0.052387,-0.232919,0.795144,0.841118,1.000000,0.279210,0.867032,0.735433,0.559150,0.182942,0.181129,0.640732,-0.220012,-0.642704,-0.677218,0.759325
carheight,0.255960,-0.541038,0.589435,0.491029,0.279210,1.000000,0.295572,0.067149,0.171071,-0.055307,0.261214,-0.108802,-0.320411,-0.048640,-0.107358,0.119336
curbweight,0.071962,-0.227691,0.776386,0.877728,0.867032,0.295572,1.000000,0.850594,0.648480,0.168790,0.151362,0.750739,-0.266243,-0.757414,-0.797465,0.835305
enginesize,-0.033930,-0.105790,0.569329,0.683360,0.735433,0.067149,0.850594,1.000000,0.583774,0.203129,0.028971,0.809769,-0.244660,-0.653658,-0.677470,0.874145
boreratio,0.260064,-0.130051,0.488750,0.606454,0.559150,0.171071,0.648480,0.583774,1.000000,-0.055909,0.005197,0.573677,-0.254976,-0.584532,-0.587012,0.553173
stroke,-0.160824,-0.008735,0.160959,0.129533,0.182942,-0.055307,0.168790,0.203129,-0.055909,1.000000,0.186110,0.080940,-0.067964,-0.042145,-0.043931,0.079443


Generating correlation heatmap 

\*note that the figure has to be closed first for the program to continue running

In [6]:
# Setting figure size
plt.figure(figsize=(10,7))

# Generating a mask to only show the bottom triangle
mask = np.triu(np.ones_like(car_df.corr(), dtype=bool))

# Generating the heatmap
sns.heatmap(car_df.corr(), annot=True, mask=mask, vmin=-1, vmax=1)
plt.title('Correlation Coefficient Of Predictors')
plt.show()

From the profile report, it can be seen that car_ID and symboling each have high cardinality. These variables used for identification purposes of the car and thus do not add predictive power to the model. Hence, I decided to discard it from the dataset.

Furthermore, the old way of defining high correlation threshold to reject variables is not applicable anymore (profile.get_rejected_variables(threshold=x)). Thus, I generated a correlation matrix and heatmap and inspect for high correlation manually. The general thumb of rule is a predictor is defined to have a collinearity if the one/many correlation score with the other variables is more than 0.8. From the matrix and the heatmap, it can be seen that highwaympg, curbweight, enginesize, horsepower, and carlength are collinear with the other variables. Thus, those variables need to be removed from the dataset.

Discarding car_ID, symboling, highwaympg, curbweight, enginesize, horsepower, and carlength from the dataset

In [7]:
car_reg = car_df.drop(columns=['car_ID','symboling','highwaympg', 'curbweight',
                               'enginesize', 'horsepower', 'carlength'])

Obtaining car make from carName (e.g. Ford F150 -> Ford)

In [8]:
car_reg['make'] = car_reg['CarName'].str.split(' ').str[0]
car_reg = car_reg.drop(columns=['CarName'])

One-hot encoding categorical predictors

In [9]:
cat_var = car_reg[['make','fueltype','aspiration','doornumber','carbody',
                   'drivewheel','enginelocation','enginetype','cylindernumber',
                   'fuelsystem']]
dummy = pd.get_dummies(cat_var, drop_first=False)

# Dropping categorical variables and concatenating the one-hot encoded variables with car_reg_train dataframe
car_reg = car_reg.drop(cat_var, axis=1)
car_reg = pd.concat([car_reg,dummy], axis=1)

Defining predictors and target variable, and dividing it into training set and test set

In [10]:
X = car_reg.drop(columns=['price'])
X = pd.DataFrame(X)
y = car_reg['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

Detecting and removing outliers using isolation forest

In [11]:
print('Before removing outliers:')
print('Xtrain = ', X_train.shape, 'ytrain = ',  y_train.shape)
print('')

# Identify outliers in the training dataset
iso = IsolationForest(contamination=0.1)
yhat = iso.fit_predict(X_train)

# Select all rows that are not outliers
mask = yhat != -1
X_train, y_train = X_train.loc[mask, :], y_train[mask]

print('After removing outliers:')
print('Xtrain = ', X_train.shape, 'ytrain = ',  y_train.shape)

Before removing outliers:
Xtrain =  (143, 74) ytrain =  (143,)

After removing outliers:
Xtrain =  (130, 74) ytrain =  (130,)


Imputing NA values with the mean of each predictors (in this dataset there is no null entries, this cell exists to take account the possibility of NA values occurences in the future)

In [12]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(X_train)
X_train = imp_mean.transform(X_train)
X_test = imp_mean.transform(X_test)

X_train = pd.DataFrame(X_train)
X_train.columns = X.columns
X_test = pd.DataFrame(X_test)
X_test.columns = X.columns

Z-standardizing the data

In [13]:
scaler = StandardScaler()

X_train_std = scaler.fit_transform(X_train)
X_train_std = pd.DataFrame(X_train_std)
X_train_std.columns = X_train.columns

X_test_std = scaler.fit_transform(X_test)
X_test_std = pd.DataFrame(X_test_std)
X_test_std.columns = X_test.columns

Selecting features for the regression task using Recursive Feature Elimination Cross Validation (RFECV)

In [14]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(max_iter=10000) # Lasso regressor for the estimator
rfecv = RFECV(estimator=lasso_reg, cv=5, n_jobs = -1)

rfecv.fit(X_train_std, y_train)

# Plotting the number of features to visualise optimum number of features
num_features = X_train_std.shape
num_features[1]

plt.switch_backend('TkAgg')
plt.figure(figsize=[10, 5])
plt.plot(range(1, num_features[1]+1), rfecv.grid_scores_)
plt.show()

# List of selected predictors (ranking = 1)
selected_predictors = pd.DataFrame(list(zip(X_train_std.columns,rfecv.ranking_)),
                                   columns = ['predictor','ranking'])

# Dropping the unselected predictors
X_train_new = X_train_std.loc[:, rfecv.support_]

## 5. Modelling

Training many quick and dirty models from different categories and cross-validating it using cross_val_score(), then display scores, mean, and standard deviation using display_scores function

In [15]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

Linear regression

In [16]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()

lin_scores = cross_val_score(lin_reg, X_train_new, y_train,
                             scoring="neg_mean_squared_error", cv=5)
display_scores(-lin_scores)

Scores: [3770682.98912948 5956287.40911231 3895149.32332885 3698316.65672149
 4158563.60110781]
Mean: 4295799.995879989
Standard deviation: 844903.243018906


Decision tree regression (CART)

In [17]:
from sklearn.tree import DecisionTreeRegressor
dt_reg = DecisionTreeRegressor(random_state=0)

dt_scores = cross_val_score(dt_reg, X_train_new, y_train,
                            scoring="neg_mean_squared_error", cv=5)
display_scores(-dt_scores)

Scores: [20075563.54700855 29338151.36992522 20313681.0579594   6908768.07799145
 10331028.65811966]
Mean: 17393438.542200856
Standard deviation: 7977514.945681946


Random forest regression

In [18]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(random_state=0)

rf_scores = cross_val_score(rf_reg, X_train_new, y_train,
                            scoring="neg_mean_squared_error", cv=5)
display_scores(-rf_scores)

Scores: [ 3577567.66407835 11145075.73794449 23670480.36422827  7755693.22280766
  7426563.84080693]
Mean: 10715076.165973138
Standard deviation: 6907151.162436673


Support vector regression (SVR)

In [19]:
from sklearn.svm import SVR
sv_reg = SVR()

sv_scores = cross_val_score(sv_reg, X_train_new, y_train,
                            scoring="neg_mean_squared_error", cv=5)
display_scores(-sv_scores)

Scores: [18796077.75013801 82144561.71125294 70840369.94023062 46754946.13323227
 64392738.30668732]
Mean: 56585738.76830824
Standard deviation: 22100226.90186989


K-nearest neighbor (K-NN) regression

In [20]:
from sklearn.neighbors import KNeighborsRegressor
knn_reg = KNeighborsRegressor()

knn_scores = cross_val_score(knn_reg, X_train_new, y_train,
                            scoring="neg_mean_squared_error", cv=5)
display_scores(-knn_scores)

Scores: [ 2330851.36307692 27374640.61692308 19469754.90153847 12872579.25076923
 19159224.05846154]
Mean: 16241410.038153846
Standard deviation: 8339512.40614074


Gradient boosting regression

In [21]:
from sklearn.ensemble import GradientBoostingRegressor
gb_reg = GradientBoostingRegressor(random_state=0)

gb_scores = cross_val_score(gb_reg, X_train_new, y_train,
                            scoring="neg_mean_squared_error", cv=5)
display_scores(-gb_scores)

Scores: [ 3933033.64666563  8792065.7036541  19686552.72508294  6166858.97929885
  7860795.29900197]
Mean: 9287861.270740699
Standard deviation: 5455821.637139364


## 6. Model Selection

Based on the MSE score, these three regression techniques perform better than the other: gradient boosting regression, K-NN regression, and random forest regression.

In terms of simplicity. The three techniques mentioned above are nowhere near simple. Those techniques are all using complex algortihms to generate the output. Furthermore, those techniques are considered as black-box algorithms, means that the process and the output of those techniques are hard to be explained and interpreted.

Because of the similarity in terms of simplicity, explainability and interpretability, the model will be selected only by looking on the MSE. Since gradient boosting has the lowest MSE, thus the technique is selected to be fine-tuned and later be used to predict the price of cars.

## 7. Model Fine-Tuning and Testing

Finding the best hyperparameter combination using RandomizedSearchCV(). Note that squared error and absolute error loss function are not included in the parameter since most of the results ended up being infinite

In [22]:
model = GradientBoostingRegressor(random_state=0)

parameters = {}
parameters['loss'] = ['huber', 'quantile']
parameters['learning_rate'] = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.5, 1]
parameters['n_estimators'] = [100, 500, 1000, 5000, 10000, 50000, 100000]
parameters['subsample'] = [0.5, 0.6, 0.7, 0.8, 0.9, 1]
              
rand_search = RandomizedSearchCV(estimator=model, param_distributions=parameters, 
                           scoring='neg_mean_squared_error', cv=3, n_iter=15, n_jobs=-1)
rand_search.fit(X_train_new, y_train)

RandomizedSearchCV(cv=3, estimator=GradientBoostingRegressor(random_state=0),
                   n_iter=15, n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.02, 0.03,
                                                          0.04, 0.05, 0.06,
                                                          0.07, 0.08, 0.09, 0.1,
                                                          0.5, 1],
                                        'loss': ['huber', 'quantile'],
                                        'n_estimators': [100, 500, 1000, 5000,
                                                         10000, 50000, 100000],
                                        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9,
                                                      1]},
                   scoring='neg_mean_squared_error')

In [23]:
cv_result = rand_search.cv_results_
for mean_score, params in zip(cv_result["mean_test_score"], cv_result["params"]):
    print(-mean_score, params)

6603843.870400612 {'subsample': 0.6, 'n_estimators': 10000, 'loss': 'quantile', 'learning_rate': 0.08}
92698162.26351814 {'subsample': 0.9, 'n_estimators': 100000, 'loss': 'huber', 'learning_rate': 0.07}
8277576.162679869 {'subsample': 0.5, 'n_estimators': 5000, 'loss': 'huber', 'learning_rate': 0.01}
7974674.480754703 {'subsample': 0.8, 'n_estimators': 100, 'loss': 'huber', 'learning_rate': 0.08}
6884651.757886429 {'subsample': 0.6, 'n_estimators': 500, 'loss': 'quantile', 'learning_rate': 0.1}
9691316.912953958 {'subsample': 1, 'n_estimators': 500, 'loss': 'huber', 'learning_rate': 0.07}
8115932.747254193 {'subsample': 0.6, 'n_estimators': 50000, 'loss': 'quantile', 'learning_rate': 0.05}
9781523.66807094 {'subsample': 0.9, 'n_estimators': 1000, 'loss': 'quantile', 'learning_rate': 0.06}
8737719.821120635 {'subsample': 0.6, 'n_estimators': 5000, 'loss': 'huber', 'learning_rate': 0.01}
14329960.406405607 {'subsample': 1, 'n_estimators': 1000, 'loss': 'quantile', 'learning_rate': 0.02}

Score when best hyperparameter combination is applied

In [24]:
-rand_search.best_score_

6603843.870400612

Best hyperparameter combination

In [25]:
rand_search.best_params_

{'subsample': 0.6,
 'n_estimators': 10000,
 'loss': 'quantile',
 'learning_rate': 0.08}

Ranking the predictors based on the feature importances

In [26]:
feature_importances = rand_search.best_estimator_.feature_importances_
sorted(zip(feature_importances, X_train_new.columns), reverse=True)

[(0.21407356978719377, 'wheelbase'),
 (0.2002163050927338, 'carwidth'),
 (0.17554269277076331, 'boreratio'),
 (0.17468380021640514, 'peakrpm'),
 (0.038214759387688424, 'make_bmw'),
 (0.02713223268272351, 'carbody_convertible'),
 (0.02655665414178465, 'make_alfa-romero'),
 (0.020541832141065985, 'aspiration_std'),
 (0.01867802498426861, 'make_jaguar'),
 (0.01711818622564527, 'cylindernumber_six'),
 (0.015049010590993278, 'enginetype_ohc'),
 (0.014967037999019982, 'cylindernumber_five'),
 (0.014872774109094645, 'cylindernumber_four'),
 (0.01195684590711941, 'make_buick'),
 (0.01107077045221442, 'enginetype_ohcv'),
 (0.009150773692896672, 'fuelsystem_1bbl'),
 (0.007422413653135311, 'make_honda'),
 (0.002752316165253836, 'cylindernumber_twelve')]

Evaluating the model on the test set

In [29]:
# Removing discarded predictors from RFECV
X_test_new = X_test_std.loc[:, rfecv.support_]

final_model = rand_search.best_estimator_
y_pred = final_model.predict(X_test_new)
final_mse = mean_squared_error(y_test, y_pred)
print('MSE of the test set:', final_mse)

MSE of the test set: 10384668.394696208


## 8. Results Discussion

\* the result (mse and feature importance) of each run may be different due to the randomness brought by the RFECV and RandomizedSearchCV. 

The model is built using gradient boosing regressor and the predictors are selected using RFECV. Then, the hyperparameters gradient boosting regressor are tuned using  When tested against the test set, the model resulted in the mean squared error score of 10,384,668. It shows that the model is able of predicting the price of car with an acceptable accuracy.

Furthermore, the feature importance can also be obtained from the model. It can be seen that wheelbase is the most important factor for determining the price, followed by carwidth, boreratio, peakrpm, and many more. This information can be used to find out the how important each predictor is to the price of cars, relative to the model.

Given that, the model has fulfilled the objectives stated on the very top of this notebook.

In [ ]:
#Predictors used in the model along with feature importance score, that contributed to the results used for this discussion:
#[(0.21407356978719377, 'wheelbase'),
# (0.2002163050927338, 'carwidth'),
# (0.17554269277076331, 'boreratio'),
# (0.17468380021640514, 'peakrpm'),
# (0.038214759387688424, 'make_bmw'),
# (0.02713223268272351, 'carbody_convertible'),
# (0.02655665414178465, 'make_alfa-romero'),
# (0.020541832141065985, 'aspiration_std'),
# (0.01867802498426861, 'make_jaguar'),
# (0.01711818622564527, 'cylindernumber_six'),
# (0.015049010590993278, 'enginetype_ohc'),
# (0.014967037999019982, 'cylindernumber_five'),
# (0.014872774109094645, 'cylindernumber_four'),
# (0.01195684590711941, 'make_buick'),
# (0.01107077045221442, 'enginetype_ohcv'),
# (0.009150773692896672, 'fuelsystem_1bbl'),
# (0.007422413653135311, 'make_honda'),
# (0.002752316165253836, 'cylindernumber_twelve')]
 
#Combination of hyperparameter that contributed to the results used for this discussion:
#{'subsample': 0.6,
# 'n_estimators': 10000,
# 'loss': 'quantile',
# 'learning_rate': 0.08}